<a href="https://colab.research.google.com/github/tanishqsrivastavaa/SCP-fine-tuning/blob/master/SCP_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
!cd /content/drive/MyDrive/

In [ ]:
import json

file_path= 'scp-api/docs/data/scp/items/content_series-1.json'

In [ ]:
with open(file_path,'r') as f:
  data= json.load(f)

In [ ]:
print(f"Data type: {type(data)}")

Data type: <class 'dict'>


In [ ]:
if isinstance(data,dict):
  first_key= list(data.keys())[0]
  print(f"\nFirst key: {first_key}")
  print(f"Sample Entry Keys: {data[first_key].keys()}")


First key: SCP-002
Sample Entry Keys: dict_keys(['created_at', 'creator', 'domain', 'history', 'hubs', 'images', 'link', 'page_id', 'rating', 'raw_content', 'raw_source', 'references', 'scp', 'scp_number', 'series', 'tags', 'title', 'url'])


In [ ]:
content_preview= data['SCP-002']['raw_content']

In [ ]:
print(content_preview[:500])

<html><body><div id="page-content">

<div class="scp-image-block block-right" style="width:300px;"><img alt="800px-SCP002-new.jpg" class="image" src="https://scp-wiki.wdfiles.com/local--files/scp-002/800px-SCP002-new.jpg"/>
<div class="scp-image-caption">
<p>SCP-002 in its containment area</p>
</div>
</div>
<p><strong>Item #:</strong> SCP-002</p>
<p><strong>Object Class:</strong> Euclid</p>
<p><strong>Special Containment Procedures:</strong> SCP-002 is to remain connected to a suitable power sup


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
raw_html= data['SCP-002']['raw_content']

soup= BeautifulSoup(raw_html,"html.parser")

In [ ]:
clean_text= soup.get_text(separator="\n")

In [ ]:
clean_text = "\n".join([line.strip() for line in clean_text.splitlines() if line.strip()])

In [ ]:
clean_text

'SCP-002 in its containment area\nItem #:\nSCP-002\nObject Class:\nEuclid\nSpecial Containment Procedures:\nSCP-002 is to remain connected to a suitable power supply at all times, to keep it in what appears to be a recharging mode. In case of electrical outage, the emergency barrier between the object and the facility is to be closed and the immediate area evacuated. Once facility power is re-established, alternating bursts of X-ray and ultraviolet light must strobe the area until SCP-002 is re-affixed to the power supply and returned to recharging mode. Containment area is to be kept at negative air pressure at all times.\nTeams including a minimum of two (2) members are required within 20 meters of SCP-002 or its containment area. Personnel should maintain physical contact with one another at all times to confirm there is another person present, as perception may be dulled, skewed, or influenced by proximity to the object.\nNo personnel below Level 3 are permitted within SCP-002. Thi

## TIME TO BATCH PROCESS

In [ ]:
import json
import glob
import os
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
file_pattern= "scp-api/docs/data/scp/items/content_series-*.json"
output_file= "scp_training_data.jsonl"

In [ ]:
files= glob.glob(file_pattern)
print(f"Found {len(files)} Series files to process")

Found 15 Series files to process


In [ ]:
with open(output_file, 'w', encoding='utf-8') as outfile:
  count=0
  for file_path in files:
    with open(file_path,'r',encoding='utf-8') as f:
      data=json.load(f)

    for scp_id,content in tqdm(data.items(),desc=f"Processing {os.path.basename(file_path)}"):
      raw_html = content.get("raw_content","")
      if not raw_html:
        continue

      soup = BeautifulSoup(raw_html,"html.parser")
      clean_text= soup.get_text(separator="\n")

      clean_text= "\n".join([line.strip() for line in clean_text.splitlines() if line.strip()])

      if len(clean_text) < 500:
                continue

      training_entry= {
          "instruction": "Write a clinical Special Containment Procedures report for the following object.",
          "input":scp_id,
          "output":clean_text
      }

      json.dump(training_entry,outfile)
      outfile.write("\n")
      count+=1


print(f"--- COMPLETE ---")
print(f"Successfully processed {count} SCP articles.")
print(f"Saved to: {output_file}")

Processing content_series-9.5.json: 100%|██████████| 398/398 [00:12<00:00, 31.41it/s]

--- COMPLETE ---
Successfully processed 9090 SCP articles.
Saved to: scp_training_data.jsonl


In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-jnyif49h/unsloth_d72903e6fd1341d7b8241fc9df4161d1
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-jnyif49h/unsloth_d72903e6fd1341d7b8241fc9df4161d1
  Resolved https://github.com/unslothai/unsloth.git to commit 33b0343ec56595d4e7d7cdd25f173207ebf991b0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 110.8 MB/s eta 0:00:00

In [ ]:
!pip install --no-deps xformers trl peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 6.9 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch

In [ ]:
max_seq_length= 2048
dtype= None
load_in_4bit= True

In [ ]:
model, tokenizer= FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length= max_seq_length,
    dtype= dtype,
    load_in_4bit=load_in_4bit,
)

print("MODEL LOADED SUCCESSFULY")

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:45: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

MODEL LOADED SUCCESSFULY


# LORA ADAPTERS
## Improving attenion: "q_proj","k_proj","v_proj","o_proj"
## Enhancing Overall Understanding: "gate_proj"
## Improve feedforward layers: "up_proj","down_proj"

In [ ]:
model= FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules= ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state= 3407,
    use_rslora=False,
    loftq_config=None,
)

print("lora adapters attached")

Unsloth 2026.1.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


lora adapters attached


In [ ]:
from datasets import load_dataset

alpaca_prompt= """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
EOS_TOKEN= tokenizer.eos_token

In [ ]:
def formatting_prompts_func(examples):
  instructions= examples["instruction"]
  inputs= examples["input"]
  outputs= examples["output"]
  texts= []

  for instruction, input,output in zip(instructions,inputs,outputs):
    text= alpaca_prompt.format(instruction,input,output) + EOS_TOKEN
    texts.append(text)
  return {"text":texts}

In [ ]:
dataset= load_dataset("json",data_files="scp_training_data.jsonl",split="train")

In [ ]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 9090
})

In [ ]:
dataset= dataset.map(formatting_prompts_func,batched=True)

print("DATA READY!!!")

print(f"sample prompt:\n{dataset[0]['text'[:300]]}")

Map:   0%|          | 0/9090 [00:00<?, ? examples/s]

DATA READY!!!
sample prompt:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a clinical Special Containment Procedures report for the following object.

### Input:
SCP-002

### Response:
SCP-002 in its containment area
Item #:
SCP-002
Object Class:
Euclid
Special Containment Procedures:
SCP-002 is to remain connected to a suitable power supply at all times, to keep it in what appears to be a recharging mode. In case of electrical outage, the emergency barrier between the object and the facility is to be closed and the immediate area evacuated. Once facility power is re-established, alternating bursts of X-ray and ultraviolet light must strobe the area until SCP-002 is re-affixed to the power supply and returned to recharging mode. Containment area is to be kept at negative air pressure at all times.
Teams including a minimum of two (2) members are requir

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer= SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset= dataset,
    dataset_text_fiel="text",
    max_seq_length=max_seq_length,
    dataset_num_proc= 2,
    packing=False,
    args= TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        #num_train_epochs=1,
        learning_rate= 2e-4,
        fp16= not torch.cuda.is_bf16_supported(),
        bf16= torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim="adamw_8bit",
        weight_decay= 0.01,
        lr_scheduler_type= "linear",
        seed= 3407,
        output_dir= "outputs",
        report_to="none",
    ),
)
trainer_stats= trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,090 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,2.290800
2,2.079700
3,2.364700
4,2.063800
5,2.128800
6,2.117700
7,1.941200
8,2.135400
9,2.095300
10,2.107900


In [ ]:
FastLanguageModel.for_inference(model)

instruction="Write a clinical Special Containment Procedures report for the following object."
input_text = "SCP-8008: A coffee mug that makes the drinker forget their own name."

prompt= alpaca_prompt.format(instruction,input_text,"")

inputs = tokenizer([prompt],return_tensors="pt").to("cuda")

print("--- GENERATING SCP REPORT ---")
outputs= model.generate(**inputs,max_new_tokens=512,use_cache=True)

result= tokenizer.batch_decode(outputs,skip_special_tokens=True)

print(result[0].split("### Reponse:")[-1])

--- GENERATING SCP REPORT ---
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a clinical Special Containment Procedures report for the following object.

### Input:
SCP-8008: A coffee mug that makes the drinker forget their own name.

### Response:
Item #:
SCP-8008
Object Class:
Safe
Special Containment Procedures:
SCP-8008 is to be contained in a standard containment locker. It is to be kept in a separate room, with the exception of use for testing.
Description:
SCP-8008 is a ceramic coffee mug, painted with the words "Good Morning!" in English. When a human being drinks from SCP-8008, they will temporarily forget their own name. This effect is instantaneous and is permanent, unless the drinker is given their name again. The name will be forgotten for 24 hours, after which the drinker will be able to recall it again.
Addendum 8008.1:
SCP-8008 was discov

In [ ]:
# 1. Save to Google Drive
# We only save the "adapters" (about 100MB), not the full 8GB model.
save_path = "/content/drive/MyDrive/scp-llama3-lora"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"--- SAVED TO GOOGLE DRIVE ---")
print(f"Location: {save_path}")

--- SAVED TO GOOGLE DRIVE ---
Location: /content/drive/MyDrive/scp-llama3-lora


In [ ]:
# Save to 4-bit GGUF (perfect for laptops)
model.save_pretrained_gguf(
    "scp_model_gguf",
    tokenizer,
    quantization_method = "q4_k_m"
)

# Move the GGUF file to Drive so you can download it later
!cp scp_model_gguf/unsloth.Q4_K_M.gguf /content/drive/MyDrive/scp-llama3-v1.gguf

print("--- EXPORT COMPLETE ---")
print("Check your Google Drive for 'scp-llama3-v1.gguf'")

Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]